In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from tqdm import tqdm
tqdm.pandas()

files = ['races']

def get_img_src(url):
    try:
        html = requests.get(url)
        soup = BeautifulSoup(html.text, 'html.parser')
        return f"http:{soup.find('table', class_='infobox').find('tbody').find('img').get('src')}"
    except:
        return ''

for i in files:
    print(i)
    df = pd.read_csv(f'{i}.csv')
    urls = df.url
    df['img_urls'] = df.url.progress_apply(get_img_src)
    df.to_csv(f'{i}.csv')


























races


100%|██████████████████████████████████████████████████████████████████████████████| 1079/1079 [13:28<00:00,  1.33it/s]


In [ ]:
import asyncio
import aiohttp
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import nest_asyncio
nest_asyncio.apply()

files = ['circuits', 'drivers', 'constructors']

async def get_img_src(session: aiohttp.ClientSession, url: str) -> str:
    """
    Асинхронно получает HTML-страницу по URL, парсит её с помощью BeautifulSoup,
    находит первый тег <img> в таблице с классом "infobox" и возвращает его URL.
    """
    try:
        async with session.get(url) as response:
            html = await response.text()
            soup = BeautifulSoup(html, 'html.parser')
            # Если структура страницы может быть иной, рекомендуется добавить проверки
            table = soup.find('table', class_='infobox')
            if table is None:
                return None
            tbody = table.find('tbody')
            if tbody is None:
                return None
            img_tag = tbody.find('img')
            if img_tag is None:
                return None
            src = img_tag.get('src')
            return f"http:{src}" if src else None
    except Exception as e:
        # Можно залогировать ошибку или вернуть None
        return None

async def get_img_src_with_progress(session: aiohttp.ClientSession, url: str, pbar: tqdm) -> str:
    """
    Обёртка для get_img_src, которая обновляет прогресс-бар после завершения запроса.
    """
    result = await get_img_src(session, url)
    pbar.update(1)
    return result

async def process_file(file: str):
    """
    Загружает данные из CSV, асинхронно обрабатывает все URL-адреса и сохраняет результат обратно в CSV.
    """
    df = pd.read_csv(f'{file}.csv')
    async with aiohttp.ClientSession() as session:
        # Инициализируем tqdm для отслеживания прогресса по количеству URL в DataFrame
        with tqdm(total=len(df), desc=f"Processing {file}") as pbar:
            # Создаём задачи для каждого URL с обёрткой, обновляющей прогресс-бар
            tasks = [
                get_img_src_with_progress(session, url, pbar)
                for url in df['url']
            ]
            # Запускаем все задачи параллельно
            results = await asyncio.gather(*tasks)
    df['img_urls'] = results
    df.to_csv(f'{file}.csv', index=False)

async def main():
    # Обрабатываем файлы последовательно; если нужно, можно запустить их параллельно:
    for file in files:
        await process_file(file)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())


Processing circuits:   4%|██▍                                                           | 3/76 [00:00<00:19,  3.79it/s]C:\Users\train\AppData\Local\Programs\Python\Python311\Lib\site-packages\bs4\builder\_htmlparser.py:116: RuntimeWarning: coroutine 'main' was never awaited
  for key, value in attrs:
Processing drivers: 100%|███████████████████████████████████████████████████████████▉| 853/854 [01:37<00:00, 10.12it/s]